In [1]:
import os
import numpy as np
import torch
from tqdm.notebook import tqdm  # Use tqdm_notebook for Jupyter Notebooks
from torch import nn
import torch.nn.functional as F  # For softmax

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 256, 128)  # 2048 -> 1024 -> 512 -> 256 after 3 pooling layers
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(1)

        x = self.pool(self.relu(self.conv1(x)))  # Output: (16, 16, 1024)
        x = self.pool(self.relu(self.conv2(x)))  # Output: (16, 32, 512)
        x = self.pool(self.relu(self.conv3(x)))  # Output: (16, 64, 256)

        x = self.flatten(x)  # Output: (16, 64 * 256)
        x = self.relu(self.fc1(x))  # Output: (16, 128)
        x = self.fc2(x)  # Output: (16, 2)

        return x

directory = "../data/recorded_data/training_psd"
model = torch.load("model.pth")
model.eval()

total_files = 0
bt_count = 0
wifi_count = 0

# Initialize tqdm_notebook with dynamic updates
with torch.no_grad():
    pbar = tqdm(os.listdir(directory), desc="Processing files", unit="file", position=0)
    for filename in pbar:
        if filename.endswith(".npy"):
            total_files += 1
            input_data = np.load(os.path.join(directory, filename))
            input_data = torch.tensor(input_data, dtype=torch.float32)
            input_data = input_data.unsqueeze(0).unsqueeze(0)
            output = model(input_data.cuda())
            
            # Apply softmax to get probabilities
            probabilities = F.softmax(output, dim=1)
            
            # Get the predicted class
            predicted_class = torch.argmax(output).item()
            
            # Get the confidence (probability) of the predicted class
            confidence = probabilities[0, predicted_class].item()

            if predicted_class == 1:
                bt_count += 1
            else:
                wifi_count += 1
            
            # Update the progress bar with both BT and WiFi counts
            pbar.set_postfix({'BT detected': bt_count, 'WiFi detected': wifi_count})

            # Optionally print the prediction and confidence for each file
            if predicted_class == 1:
                print(f"{filename} was classified as a BT with confidence {confidence*100:.2f}%")
            else:
                print(f"{filename} was classified as WiFi with confidence {confidence*100:.2f}%")

bt_percentage = (bt_count / total_files) * 100
wifi_percentage = ((total_files - bt_count) / total_files) * 100

print(f"\nClassification Summary:")
print(f"Total files processed: {total_files}")
print(f"Bluetooth devices detected: {bt_count} ({bt_percentage:.2f}%)")
print(f"WiFi devices detected: {wifi_count} ({wifi_percentage:.2f}%)")


/tmp/ipykernel_27028/1596577314.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pth")


Processing files:   0%|          | 0/25082 [00:00<?, ?file/s]

wifi_psd_wifi_ch1_1.npy_555.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch2_2.npy_387.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_2.npy_252.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch10_1.npy_686.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch1_4.npy_342.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_5.npy_823.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch4_2.npy_366.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_3.npy_666.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch2_2.npy_346.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_1.npy_210.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch1_1.npy_96.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_2.npy_463.npy was classified as WiFi with confidence 100.00%
wifi_psd_wifi_ch11_3.npy_202.npy was classified